# Clean Indatabet-Free-Odds Data

In [1]:
import pathlib
import sys
import datetime

import numpy as np
import pandas as pd
from pyxlsb import open_workbook as open_xlsb
import missingno as msno
import matplotlib.pyplot as plt

%matplotlib inline

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
# import my class code from the source
# %aimport src-dir.filename

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PROJECT_DIR = pathlib.Path().cwd().resolve().parent

SCOPED_DATA_DIR = PROJECT_DIR / 'data' / '02-scoped'
CLEANED_DATA_DIR = PROJECT_DIR / 'data' / '03-cleaned'

In [2]:
def drop_unnamed(df_orig):
    """
    COMMON !!!!!!!!!!!!!
    """
    df = df_orig.copy(deep=True)
    drop_cols = [col for col in df.columns if 'Unnamed' in col]
    df.drop(columns = drop_cols, inplace=True)    
    return df

def drop_all_nulls(df_orig, axis=1):
    """
    COMMON !!!!!!!!!
    """
    df = df_orig.copy(deep=True)
    # Drop any columns that are all null
    df = df_orig.dropna(axis=axis, how='all')
    return df

def collect_col_names(df_origs):
    """
    COMMON UTILITY !!!!!!!!!!
    """
    cols = []
    for df in df_origs:
        cols.extend(list(df.columns))
    return set(cols)

def rename_indatabet_cols(df_orig):
    """
    """
    df = df_orig.copy(deep=True)
    
    odds_cols = {'odds_awin_pinn': 'awinOddsPinnIndatabet',
                 'odds_draw_pinn': 'drawOddsPinnIndatabet',
                 'odds_hwin_pinn': 'hwinOddsPinnIndatabet',
                 'odds_awin_bet365': 'awinOddsBet365Indatabet',
                 'odds_draw_bet365': 'drawOddsBet365Indatabet',
                 'odds_hwin_bet365': 'hwinOddsBet365Indatabet',
                 'odds_ftgoalso2.5_bet365': 'ftGoalsO2.5OddsBet365Indatabet',
                 'odds_ftgoalsu2.5_bet365': 'ftGoalsU2.5OddsBet365Indatabet',
                 'odds_ftgoalso2.5_pinn': 'ftGoalsO2.5OddsPinnIndatabet',
                 'odds_ftgoalsu2.5_pinn': 'ftGoalsU2.5OddsPinnIndatabet'}
    
    df.rename(columns=odds_cols, inplace=True)
    return df


def make_results_col(df_orig):
    """
    COMMON !!!!!!!!!!!!!!!!!!!!!!!!
    """
    df = df_orig.copy(deep=True)
    try:
        if 'result' in df.columns:
            df.drop(columns=['result'], inplace=True)
            # Calculate Results column
        conditions = [df['h_ftGoals'] > df['a_ftGoals'],
                      df['h_ftGoals'] == df['a_ftGoals'],
                      df['h_ftGoals'] < df['a_ftGoals']]
        choices = ['hwin', 'draw', 'awin']
        df['result'] = np.select(conditions, choices, default='not-played')
    except:
        # Where there are abandoned matches or penalty finishes this fails
        df['result'] = None
    return df  

def lowercase_team_names(df_orig):
    """
    COMMON !!!!!!!!!!!!!!
    """
    df = df_orig.copy(deep=True)
    for col in ['h', 'a']:
        df[col] = df[col].str.lower().str.replace(' ', '-').str.replace('.','')
    return df
    
    
    
def get_fps(top_level_dir, ext='csv'):
    """
    COMMON !!!!!!!!!!!
    """
    src_fps = list(top_level_dir.rglob('*.' + ext))
    return src_fps

def read_csvs(fps):
    """
    COMMON !!!!!!!!!!!!!!!
    
    """
    dfs = [pd.read_csv(fp) for fp in fps]
    return dfs

def clean_up_dfs(dfs):
    clean_dfs = []
    for df in dfs:
        df = drop_unnamed(df)
        df = drop_all_nulls(df, axis=1)
        df = drop_all_nulls(df, axis=0)
        df = rename_indatabet_cols(df)
        df = make_results_col(df)
        df = lowercase_team_names(df)
        clean_dfs.append(df)
    return clean_dfs


def make_save_fps(top_level_dir, season_dfs, source = 'indatabet-com'):
    """
    COMMON !!!!!!!!!!!!
    """
    
    scoped_fdcuk_fps = []
    for season_df in season_dfs:
        nation = season_df['nation'].unique()[0]
        league = season_df['league'].unique()[0]
        season = season_df['season'].unique()[0]
        fn = str(season) + '.csv'
        save_fp = top_level_dir / source / nation / league / season / fn
        scoped_fdcuk_fps.append(save_fp)
    return scoped_fdcuk_fps


def save_dfs_to_fps(dfs, fps):
    """
    COMMON !!!!!!!!!!!!!
    """
    n = 0
    for df, fp in zip(dfs, fps):
        if not fp.exists():
            fp.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(fp, index=False)
        n += 1
    return n


scoped_fps = get_fps(SCOPED_DATA_DIR / 'indatabet-com', ext='csv')
dfs = read_csvs(scoped_fps)
clean_dfs = clean_up_dfs(dfs)
cleaned_indatabet_fps = make_save_fps(CLEANED_DATA_DIR,
                                           clean_dfs,
                                           source = 'indatabet-com')
n_saved = save_dfs_to_fps(clean_dfs, cleaned_indatabet_fps)
n_saved

# cols = collect_col_names(clean_dfs)
# cols

165

In [3]:
clean_dfs[150].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 22 columns):
date                              180 non-null object
id_fifa                           180 non-null object
nation                            180 non-null object
league                            180 non-null object
season                            180 non-null object
h                                 180 non-null object
a                                 180 non-null object
h_htgoals                         180 non-null float64
a_htgoals                         180 non-null float64
h_ftGoals                         180 non-null float64
a_ftGoals                         180 non-null float64
hwinOddsPinnIndatabet             180 non-null float64
drawOddsPinnIndatabet             180 non-null float64
awinOddsPinnIndatabet             180 non-null float64
hwinOddsBet365Indatabet           180 non-null float64
drawOddsBet365Indatabet           180 non-null float64
awinOddsBet365Indatabe

In [4]:
stop

NameError: name 'stop' is not defined

## Compile Premier League Data

In [ ]:
ODDS_DATA_FP = RAW_DATA_DIR / 'indatabet-free-download' / 'oOo FT_2in1_Pinnacle & bet365_ML TG_01 April 2019.xlsb'

print(ODDS_DATA_FP)
save_file_name = 'indatabet-free-odds.csv'

In [ ]:
# https://stackoverflow.com/questions/45019778/read-xlsb-file-in-pandas-python
# Open xlsb file format
df = []

with open_xlsb(ODDS_DATA_FP) as wb:
    with wb.get_sheet(1) as sheet:
        for row in sheet.rows():
            df.append([item.v for item in row])

odds = pd.DataFrame(df[1:], columns=df[0])
odds.columns = odds.columns.str.lower()
odds.head()

In [ ]:
# Lots of empty cloumns to remove
odds_df = odds.copy(deep=True)

# Cutoff columns
odds_df = odds_df.iloc[:, 0:35]
# make column names strings
col_mapper = {col: str(col) for col in odds_df.columns}
odds_df.rename(columns = col_mapper, inplace=True)
# Drop unnecessary columns
# Columns that can be calculated from other data such as Results goals over, under etc
# col 12 is first None | R | none
drop_cols = set([12,13,14,15, 16, 17, 20, 21, 26, 30])
all_cols = set(range(0,odds_df.shape[1]))
keeper_cols = all_cols.difference(drop_cols)
odds_df = odds_df.iloc[:, list(keeper_cols)]
col_names = ['yy', 'dd', 'mm', 'date', 'id_fifa', 'country', 'league', 'season', 'h', 'a',
            'h_htgoals', 'a_htgoals', 'h_ftGoals', 'a_ftGoals', 'et_pen_awd',
             'odds_hwin_pinn', 'odds_draw_pinn', 'odds_awin_pinn',
            'odds_hwin_bet365', 'odds_draw_bet365', 'odds_awin_bet365',
            'odds_ftgoalso2.5_pinn', 'odds_ftgoalsu2.5_pinn',
            'odds_ftgoalso2.5_bet365', 'odds_ftgoalsu2.5_bet365']
odds_df.columns = col_names
odds_df = odds_df.iloc[2:].reset_index(drop=True)
# Put seasons into my format
odds_df['season'] = odds_df['season'].str.replace('/', '-')
# Assemble date into common format - original date column cannot be trusted - multiple formats 
odds_df['date'] = pd.to_datetime(odds_df['yy'].astype(int).astype(str) + '-' + odds_df['mm'].astype(str)
                                 + '-' + odds_df['dd'].astype(int).astype(str))
odds_df.drop(columns=['yy', 'dd', 'mm'], inplace=True)
odds_df.sort_values(by='date', ascending=True, inplace=True)
odds_df.reset_index(drop=True, inplace=True)

odds_df.head()

In [ ]:
# Format column data to lower case strings, replace  space with dash
str_cols = odds_df.columns[(odds_df.applymap(type) == str).all(0)]
print(str_cols)
for col in str_cols:
    odds_df[col] = odds_df[col].str.strip().str.lower().str.replace(' ', '-')
odds_df.head()

In [ ]:
odds_df.info()

### Change Values to match my standard names for leagues, nation

In [ ]:
odds_df['country'].unique()

In [ ]:
df_1 = odds_df.copy(deep=True)
df_1.loc[df_1['country'] == 'england', 'country'] = 'united-kingdom'
df_1['country'].unique()

In [ ]:
df_1[df_1['country'] == 'united-kingdom']['league'].unique()

In [ ]:
df_2 = df_1.copy(deep=True)
crit1 = df_2['country'] == 'united-kingdom'
crit2 = df_2['league'] == 'premier-league'
df_2.loc[crit1 & crit2, 'league'] = 'english-premier-league'

crit2 = df_2['league'] == 'championship'
df_2.loc[crit1 & crit2, 'league'] = 'english-championship'

crit2 = df_2['league'] == 'league-one'
df_2.loc[crit1 & crit2, 'league'] = 'one'

df_2[df_2['country'] == 'united-kingdom']['league'].unique()

In [ ]:
df_2[df_2['country'] == 'germany']['league'].unique()

In [ ]:
df_3 = df_2.copy(deep=True)
crit1 = df_3['country'] == 'germany'
crit2 = df_3['league'] == '2.-bundesliga'
df_3.loc[crit1 & crit2, 'league'] = 'bundesliga-2'

df_3[df_3['country'] == 'germany']['league'].unique()

In [ ]:
df_3[df_3['country'] == 'spain']['league'].unique()

In [ ]:
df_4 = df_3.copy(deep=True)
crit1 = df_4['country'] == 'spain'
crit2 = df_4['league'] == 'primera-division'
df_4.loc[crit1 & crit2, 'league'] = 'la-liga'

df_4[df_4['country'] == 'spain']['league'].unique()

In [ ]:
nations = ['germany', 'united-kingdom', 'spain', 'italy',
          'france', 'united-kingdom', 'germany', 'netherlands',
          'russian-federation', 'scotland', 'portugal', 'switzerland',
          'belgium', 'turkey', 'poland', 'united-kingdom']

# poland, switzerland are multileague

leagues = ['bundesliga', 'english-premier-league', 'la-liga', 'serie-a',
           'ligue-1', 'english-championship', 'bundesliga-2', 'eredivisie',
           'premier-league', 'premiership', 'primeira-liga', 'super-league',
           'first-division-a', 'super-lig', 'ekstraklasa', 'one']

seasons = ['2000-2001', '2001-2002', '2002-2003', '2003-2004',
           '2004-2005', '2005-2006', '2006-2007', '2007-2008',
           '2008-2009', '2009-2010', '2010-2011', '2011-2012',
           '2012-2013', '2013-2014', '2014-2015', '2015-2016',
           '2016-2017', '2017-2018']

In [ ]:
df_5 = df_4.copy(deep=True)
season_dfs = []
for nation, league in zip(nations, leagues):
    for season in seasons:
        crit1 = df_5['country'] == nation
        crit2 = df_5['league'] == league
        crit3 = df_5['season'] == season
        season_df = df_5[crit1 & crit2 & crit3]
        if len(season_df):
            season_dfs.append(season_df)

full_df = pd.concat(season_dfs, axis=0)
full_df = full_df.drop(columns=['h_htgoals', 'a_htgoals'])
full_df.rename(columns={'country': 'nation'}, inplace=True)
full_df.head()

In [ ]:
full_df.info()

In [ ]:
sub_dfs = []
full_df1 = full_df.copy(deep=True)
for (nation, league, season), df in full_df1.groupby(by=['nation', 'league', 'season']):
    for col in df.columns:
        if col not in ['date', 'id_fifa', 'nation', 'league', 'season', 'h', 'a', 'et_pen_awd']:
            #print(col, df[col].apply(np.isreal).all())
            if df[col].apply(np.isreal).all():
                df[col] = pd.to_numeric(df[col])
    if df['et_pen_awd'].notnull().sum() == 0:
        sub_dfs.append(df)
            
full_df2 = pd.concat(sub_dfs, axis=0)
full_df2.head()
    #break
#         if df[['HST', 'AST']].isnull().sum().sum() == 0:
#           st_dfs.append(df)

In [ ]:
full_df2.info()

In [ ]:
msno.matrix(full_df2)
plt.show();

In [ ]:
full_df3 = full_df2.copy(deep=True)
full_df3.drop(columns=['et_pen_awd', 'odds_ftgoalso2.5_pinn', 'odds_ftgoalsu2.5_pinn',
                       'odds_ftgoalso2.5_bet365', 'odds_ftgoalsu2.5_bet365'], inplace=True)

In [ ]:
full_df3.info()

In [ ]:
for (nation, league, season), seas_df in full_df3.groupby(by=['nation', 'league', 'season']):
    season_df = seas_df.copy(deep=True)
    season_df.sort_values(by=['date'], inplace=True)
    season_df.reset_index(drop=True, inplace=True)
    fn = season + '.csv'
    source = 'indatabet-com'
    save_dir = CLEANED_DIR / source / nation / league / season
    save_fp = save_dir / fn
    save_dir.mkdir(parents=True, exist_ok=True)
    season_df.to_csv(save_fp, index=False)

In [ ]:
season_df.head()

In [ ]:
season_df.tail()

In [ ]:
season_df.info()

In [ ]:
season_df.describe()

### Test Read

In [ ]:
df = pd.read_csv(save_fp, parse_dates=['date'], index_col=None)
df.head()

In [ ]:
df.info()

In [ ]:
stop

In [ ]:
full_df1 = full_df.copy(deep=True)
for col in full_df.columns:
    if col not in ['date', 'id_fifa', 'country', 'league', 'season', 'h', 'a', 'et_pen_awd']:
        print(col)
        full_df1[col] = full_df[col].astype(float)
    
full_df1.info()

In [ ]:
msno.matrix(full_df)
plt.show();

In [ ]:
full_df.drop(columns=[''])

In [ ]:
stop

In [ ]:
df_4[df_4['country'] == 'poland']['league'].unique()

In [ ]:
stop

In [ ]:
odds_df[odds_df['country'] == 'germany']['league'].value_counts()

In [ ]:
odds_df.to_csv(INTERIM_DATA_DIR / 'indatabet-free-odds' / save_file_name, index=False)

In [ ]:
dtypes = {'id_fifa': np.object, 'country': np.object, 'league': np.object,
          'season': np.object, 'h': np.object, 'a': np.object,
          'h_htgoals': np.float64, 'a_htgoals': np.float64,
          'h_ftgoals': np.object, 'a_ftgoals': np.object, 'et_pen_awd': np.object,
          'odds_hwin_pinn': np.float64, 'odds_draw_pinn': np.float64, 'odds_awin_pinn': np.float64,
          'odds_hwin_bet365': np.float64, 'odds_draw_bet365': np.float64, 'odds_awin_bet365': np.float64,
          'odds_ftgoalso2.5_pinn': np.float64, 'odds_ftgoalsu2.5_pinn': np.float64,
          'odds_ftgoalso2.5_bet365': np.float64, 'odds_ftgoalsu2.5_bet365': np.float64}
odds_df = pd.read_csv(INTERIM_DATA_DIR / 'indatabet-free-odds' / save_file_name,
                      parse_dates=['date'], dtype=dtypes, dayfirst=True, index_col=None)
odds_df.head()

In [ ]:
odds_df.info()

In [ ]:
odds_df.describe()